In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import boto3
import requests
import json
from datetime import datetime, timedelta

In [28]:
pratiques_path = "pratiques.csv"
locations_path = "Counterlist-DE.csv"
pratiques_df = pd.read_csv(pratiques_path,sep='\t')
locations_df = pd.read_csv(locations_path,sep='\t')

In [52]:

unique_date = []
data_sets = []

def create_request():
    json_data={}
  
    for index, row in locations_df.iterrows():
        url = 'http://data.eco-counter.com/ParcPublic/CounterData'
        #start_day, start_month, start_year = row.debut.split('/')
        yesterda_day, yesterday_month, yesterday_year = (datetime.today() - timedelta(1)).strftime('%d/%m/%Y').split('/')
        today_day, today_month, today_year = datetime.today().strftime('%d/%m/%Y').split('/')
        pratiques = ""
        if hasattr(row, 'pratiques'):
            pratiques="&pratiques="+row.pratiques
        body = "idOrganisme=4586&idPdc={}&fin={}%2F{}%2F{}&debut={}%2F{}%2F{}&interval=4&pratiques={}".format(row.idPdc, today_day, today_month, today_year, yesterda_day, yesterday_month, yesterday_year, pratiques)

        headers = {
            "Accept": "text/plain, */*; q=0.01",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "en-US,en;q=0.9",
            "Connection": "keep-alive",
            "Content-Length": "115",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Cookie": "i18next=en_US; _ga=GA1.2.1682226698.1584790632; _gid=GA1.2.220973166.1584790632",
            "Host": "data.eco-counter.com",
            "Origin": "http://data.eco-counter.com",
            "Referer": "http://data.eco-counter.com/ParcPublic/?id=4586",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        }
        x = requests.post(url, body, headers=headers)   

        for entry in x.json()[:-1]:
            data_set={}
            date = str(entry[0]) 
            formatted_date = datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d')   
            data_set['date']= formatted_date
            data_set['bike_count']= str(entry[1])
            data_set['name']= row['nom']
            data_set['lon']=row['lon']
            data_set['lat']=row['lat']
            data_sets.append(data_set)
            if formatted_date not in unique_date:
                unique_date.append(formatted_date)
    return data_sets

In [53]:
json_data = create_request()

In [55]:
df_new = pd.DataFrame(data=json_data)

In [57]:
# add a column that contains the station_string
def f(x):
    """ return the station string """
    return "('%s', '%s')" % (x["lon"], x["lat"])
df_new["station_string"] = df_new.apply(f, axis=1)
df_new["date"] = df_new.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


In [59]:
# load bike prediction modul
from prediction import BikePrediction
BT = BikePrediction()

In [67]:
# set predictions for each station by filtering df_new accordingly
stations = df_new["name"].unique()

df_enriched = pd.DataFrame()

for station in stations:
    #print("current station", station)
    tmp = df_new.loc[df_new["name"]==station]
    #print("len tmp", len(tmp))

    if len(tmp) > 450:
        # only predict if more than 365 days exist
        predictions = BT.predict_series(
            station_string=station,
            days=tmp["date"])
        
        tmp = tmp.join(predictions.set_index("date"), on="date")
    else:
        tmp["prediction"] = 0
    if len(df_enriched) == 0:
        df_enriched = tmp
    else:
        #print('ln temp', len(tmp))
        #print('ln enr', len(df_enriched))
        df_enriched = df_enriched.append(tmp)
    #df_new[df_new["station_string"]==station] = tmp

C:\Users\gabriel.jaeger\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [61]:
df_new = df_enriched

In [62]:
df_new['date']=df_new.date.apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

In [63]:
whole_json = df_new.to_json(orient='records')

In [65]:
s3_client = boto3.client('s3', 
    aws_access_key_id=settings.AWS_SERVER_PUBLIC_KEY, 
    aws_secret_access_key=settings.AWS_SERVER_SECRET_KEY, 
    region_name=REGION_NAME
    )

In [66]:

for date in unique_date:
    year=date[0:4]
    month=date[5:7]
    day=date[8:10]
    data_array=[]
    
    for i in json.loads(whole_json):
        if i['date'] == date:
            data_array.append(i)
    response = s3_client.put_object(
      Bucket="sdd-s3-basebucket",
      Body=json.dumps(data_array),     
      Key="fahrrad/"+year+"/"+month+"/"+day+'/{}.json'.format(year + '-' + month + '-' + day)
    )
    